### BP to LF transition:
#### Variables
For each $e\in E$ and every time $t$:  
-  $requested(e,t)$        
-  $blocked(e,t)$  
- $triggered(e,t)$  



For each state $s\in S(bt)$ of every b-thread $bt$ and every time $t$:

    
- $at(s,t)$  

_________________________________________________________________
#### Rules

$\forall t \colon  \oplus_{e\in E}(triggered(e,t))$

$\forall t,e \colon  triggered(e,t) \Rightarrow requested(e,t) \wedge \neg blocked(e,t)$

$\forall bt, t \colon  \oplus_{s\in S(bt)}(at(s,t))$


$at(s,t) \iff ( t=0 \wedge s \in INITIAL)  \vee (at(s',t-1) \wedge triggered(e,t-1) \wedge (s',e,s)\in TRANSITIONS)$

$requested(e,t) \iff \bigvee_{s \in S} e \in REQUESTED(s) \wedge at(s,t)$

$blocked(e,t) \iff \bigvee_{s \in S} e \in BLOCKED(s) \wedge at(s,t)$


In [55]:
import os
import re
import yaml
from yaml import CLoader as Loader, CDumper as Dumper
from collections import defaultdict

### Parse BThreads yaml

In [11]:
r = re.compile("bt\d.yaml")
bthreads_files = list(filter(r.match, os.listdir('.')))
bthreads_files

['bt1.yaml', 'bt3.yaml', 'bt2.yaml']

In [208]:
bt_dict = dict()
bthreads = dict()
for bt_file in bthreads_files:
    with open(bt_file, 'r') as bt_yaml:
        bt_dict[bt_file.split('.')[0]] = yaml.load(bt_yaml, Loader=Loader)
# bt_dict

In [248]:
INITIALS = set()
EVENTS = set()
STATES = set()
BLOCKED = defaultdict(list)
REQUESTED = defaultdict(list)
TIME = [i for i in range(3)]
TRANSITIONS = set()
for bt in bt_dict:
    INITIALS.add(f"{bt}_{bt_dict[bt]['initial']}")
    for step in bt_dict[bt]['states']:
        step_id = f"{bt}_{step['id']}"
        for key, values in step.items():
            if key == 'id':
                STATES.add(step_id)
            if key in ('R', 'W', 'B'):
                for val in values:
                    EVENTS.add(val)
              
        if len(step['B']) > 0:
            BLOCKED[step_id].extend(step['B'])
        if len(step['R']) > 0:
            REQUESTED[step_id].extend(step['R'])
    for t in bt_dict[bt]['transitions']:
        TRANSITIONS.add((f"{bt}_state_{t['source']}",f"{bt}_state_{t['target']}",t['event']))

In [249]:
INITIALS

{'bt1_state_0', 'bt2_state_0', 'bt3_state_0'}

In [250]:
STATES 

{'bt1_state_0',
 'bt1_state_1',
 'bt1_state_2',
 'bt1_state_3',
 'bt2_state_0',
 'bt2_state_2',
 'bt2_state_3',
 'bt3_state_0',
 'bt3_state_1',
 'bt3_state_2',
 'bt3_state_3'}

In [251]:
EVENTS 

{'g1', 'g2', 'g3', 'h1', 'h2', 'h3', 'o1', 'o2', 'o3'}

In [252]:
BLOCKED 

defaultdict(list,
            {'bt3_state_1': ['o1'],
             'bt3_state_2': ['o3'],
             'bt2_state_0': ['o2'],
             'bt2_state_2': ['o1'],
             'bt2_state_3': ['o3']})

In [253]:
REQUESTED

defaultdict(list,
            {'bt1_state_0': ['g1', 'g2', 'g3'],
             'bt1_state_1': ['o1', 'o2', 'o3'],
             'bt1_state_2': ['h1', 'h2', 'h3']})

In [254]:
TIME

[0, 1, 2]

In [255]:
TRANSITIONS

{('bt1_state_0', 'bt1_state_1', 'g1'),
 ('bt1_state_0', 'bt1_state_1', 'g2'),
 ('bt1_state_0', 'bt1_state_1', 'g3'),
 ('bt1_state_1', 'bt1_state_3', 'o1'),
 ('bt1_state_1', 'bt1_state_3', 'o2'),
 ('bt1_state_1', 'bt1_state_3', 'o3'),
 ('bt1_state_2', 'bt1_state_0', 'h1'),
 ('bt1_state_2', 'bt1_state_0', 'h2'),
 ('bt1_state_2', 'bt1_state_0', 'h3'),
 ('bt2_state_0', 'bt2_state_1', 'h2'),
 ('bt2_state_0', 'bt2_state_2', 'h1'),
 ('bt2_state_0', 'bt2_state_3', 'h3'),
 ('bt3_state_0', 'bt3_state_1', 'g1'),
 ('bt3_state_0', 'bt3_state_2', 'g3'),
 ('bt3_state_0', 'bt3_state_3', 'g2')}

### Literals creation

In [172]:
import aiger

In [217]:
atoms = dict()
def get_atom(name):
    if name not in atoms:
        atoms[name] = aiger.atom(name)
    return atoms[name]
        

1. $\forall t \colon  \oplus_{e\in E}(triggered(e,t))$

In [256]:
def unique_event(events=EVENTS , time=TIME):
    triggered = [get_atom(f'triggered_{e}_at_{t}') for e in events for t in time]
    formula = False

    for t in triggered:
        c = t
        for tt in triggered:
            if t != tt:
                c = c & ~tt
        formula = c | formula
    return formula
  

2. $\forall t,e \colon  triggered(e,t) \Rightarrow requested(e,t) \wedge \neg blocked(e,t)$
 

In [257]:
def bp(events=EVENTS , time=TIME):
    literals = [(get_atom(f'requested_{e}_at_{t}'),
                  get_atom(f'blocked_{e}_at_{t}'),
                  get_atom(f'triggered_{e}_at_{t}')) for e in events for t in time]
    formula = True
    for r,b,t in literals:
        formula = t.implies(r & ~b) & formula
        
    return formula

3. $\forall bt, t \colon  \oplus_{s\in S(bt)}(at(s,t))$  

In [258]:
def one_state_at_time(states=STATES , time=TIME):
    literals = [get_atom(f'{s}_at_{t}') for s in states for t in time]
    formula = False
    for l in literals:
        c = l
        for ll in literals:
            if l != ll:
                c = c & ~ll
        formula = c | formula
    return formula

4. $at(s,t) \iff ( t=0 \wedge s \in INITIAL)  \vee (at(s',t-1) \wedge triggered(e,t-1) \wedge (s',e,s)\in TRANSITIONS)$  

In [259]:
def check_transitions(states=STATES , time=TIME, initials=INITIALS, transitions=TRANSITIONS):
    at_state = [(get_atom(f'{s}_at_{t}'), s) for s in states for t in time]
    triggered = [get_atom(f'triggered_{e}_at_{t}') for e in events for t in time]
                 
                 
                 for e in events for t in time]
    formula = True
    for r,b,t in literals:
        formula = t.implies(r & ~b) & formula
        
    return formula

SyntaxError: invalid syntax (<ipython-input-259-2c7bcdaafe28>, line 6)

5. $requested(e,t) \iff \bigvee_{s \in S} e \in REQUESTED(s) \wedge at(s,t)$

In [277]:
def get_requested(events=EVENTS, states=STATES , time=TIME):
    at_state = [(get_atom(f'{s}_at_{t}'), s) for s in states for t in time]
    requested = [(get_atom(f'requested_{e}_at_{t}'),e) for e in events for t in time]
    formula = True
    a= list()
    for req, e in requested:
        states_disjunction = False
        for state, s in at_state:
            if e in REQUESTED[s]:
                states_disjunction = state |states_disjunction
        formula = req.implies(states_disjunction) & states_disjunction.implies(req) & formula
    return formula

In [278]:
get_requested()

BoolExpr(_aig=aag 161 36 0 1 125
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
322
74 17 15
76 74 19
78 76 54
80 77 55
82 81 79
84 5 3
86 84 7
88 87 31
90 86 30
92 91 89
94 86 24
96 87 25
98 97 95
100 5 3
102 100 7
104 102 22
106 103 23
108 107 105
110 100 7
112 111 37
114 110 36
116 115 113
118 111 35
120 110 34
122 121 119
124 84 7
126 124 32
128 125 33
130 129 127
132 11 9
134 132 13
136 135 73
138 134 72
140 139 137
142 11 9
144 142 13
146 144 70
148 145 71
150 149 147
152 77 43
154 76 42
156 155 153
158 74 19
160 158 40
162 159 41
164 163 161
166 77 39
168 76 38
170 169 167
172 17 15
174 172 19
176 174 48
178 175 49
180 179 177
182 172 19
184 182 44
186 183 45
188 187 185
190 159 47
192 158 46
194 193 191
196 194 188
198 196 180
200 134 56
202 135 57
204 203 201
206 204 198
208 132 13
210 208 58
212 209 59
214 213 211
216 214 206
218 145 61
220 144 60
222 221 219
224 222 216
226 224 170
228 226 164
230 228 156
232 144 68
23

6. $blocked(e,t) \iff \bigvee_{s \in S} e \in BLOCKED(s) \wedge at(s,t)$  

In [ ]:
def get_blocked(events=EVENTS, states=STATES , time=TIME):
    at_state = [(get_atom(f'{s}_at_{t}'), s) for s in states for t in time]
    blocked = [(get_atom(f'blocked{e}_at_{t}'),e) for e in events for t in time]
    formula = True
    a= list()
    for block, e in blocked:
        states_disjunction = False
        for state, s in at_state:
            if e in REQUESTED[s]:
                states_disjunction = state |states_disjunction
        formula = block.implies(states_disjunction) & states_disjunction.implies(block) & formula
    return formula

In [176]:
formula = (unique_event() &bp())
cnf = CNF(from_aiger=(formula.aig))
cnf.to_file('cnf_bp.cnf')
cnf2 = aig2cnf(unique_event().aig)


In [177]:
len(cnf2.clauses)

81